### Starting the Game Project
- In pygame, the origin (0,0) is at the top-left corner of the screen, and coordinates increase as you go down and to the right. On a 1200 by 800 screen, the origin is at the top-left corner, and the bottom-right corner has the coordinates(1200,800).
- Collisions happen when game elements overlap
    - sprite.groupcollide() looks for collisions between members of two groups
- Notice that we never make more than on ship; we make only oe ship instance for the whole game and recenter it whenever the ship has been hit. The statistic ship_left will tell us when the player has run out of ships.
- blit -> block image transfer

In [ ]:
import sys
import pygame
from pygame.sprite import Group

def run_game():
    # Initialize pygame, settings, and screen object.
    pygame.init() # initialize background settings that Pygame needs to work properly
    ai_settings = Settings() # create an instance of Settings to be used throughout game
    
    # creates a display window (or surface)
    screen = pygame.display.set_mode((ai_settings.screen_width, ai_settings.screen_height)) 
    pygame.display.set_caption("Alien Invasion")
    
    # Make the Play button.
    play_button = Button(ai_settings, screen, "Play")
    
    # Create an instance to store game statistics and create a scoreboard.
    stats = GameStats(ai_settings)
    sb = Scoreboard(ai_settings, screen, stats)
    
    # Make a ship.
    ship = Ship(ai_settings, screen)
    
    # Make a group to store bullets in.
    bullets = Group() # instance of the class pygame.sprite.Group
    
    # Make a group of aliens
    aliens = Group()
    
    # Create the fleet of aliens
    create_fleet(ai_settings, screen, ship, aliens)
    
    # Start the main loop for the game.
    while True:
        check_events(ai_settings, screen, stats, sb, play_button, ship, aliens, bullets)
        
        # Runs only when game is active
        if stats.game_active:
            ship.update()
            update_bullets(ai_settings, screen, stats, sb, ship, aliens, bullets)
            update_aliens(ai_settings, screen, stats, sb, ship, aliens, bullets)
        update_screen(ai_settings, screen, stats, sb, ship, aliens, bullets, play_button)
        

run_game()

In [ ]:
quit

In [ ]:
# Creating a Settings Class
class Settings():
    """A class to store all settings for Alien Invasion."""
    
    def __init__(self):
        """Initialize the game's static settings."""
        # Screen settings
        self.screen_width = 1200
        self.screen_height = 800
        self.bg_color = (230, 230, 230)
        
        # Ship settings
        self.ship_speed_factor = 1.5 # number of pixels adjusted
        self.ship_limit = 3 # number of lives + starting life
        
        # Bullet settings
        self.bullet_speed_factor = 3
        self.bullet_width = 3
        self.bullet_height = 15
        self.bullet_color = 60, 60, 60
        self.bullets_allowed = 3
        
        # Alien settings
        self.alien_speed_factor = 1
        self.fleet_drop_speed = 10
        # fleet direction of 1 represents right; -1 represent left.
        self.fleet_direction = 1
        
        # How quickly the game speeds up after every level completion
        self.speedup_scale = 1.4
        
        # How quickly the alien point values increases after each level
        self.score_scale = 2.0
        
        # Initialize values for attributes needed to change throughout the course of the game
        self.initialize_dynamic_settings()
        
    def initialize_dynamic_settings(self):
        """Initialize settings that change throughout the game."""
        self.ship_speed_factor = 1.5
        self.bullet_speed_factor = 3
        self.alien_speed_factor = 1
        
        # fleet direction of 1 represents right; -1 represent left.
        self.fleet_direction = 1
        
        # Scoring
        self.alien_points = 50
    
    def increase_speed(self):
        """Increase speed settings and alien point values.."""
        self.ship_speed_factor *= self.speedup_scale
        self.bullet_speed_factor *= self.speedup_scale
        self.alien_speed_factor *= self.speedup_scale
        
        self.alien_points = int(self.alien_points * self.score_scale)
        print(self.alien_points) # prints out the worth of an alien at each level
        

In [ ]:
# Creating the Ship Class
from pygame.sprite import Sprite

class Ship(Sprite):
    
    def __init__(self, ai_settings, screen):
        """Initialize the ship and set its starting position."""
        super(Ship, self).__init__() # ensure that ship inherits from Sprite
        self.screen = screen
        self.ai_settings = ai_settings # create attribute
        
        # Load the ship image and get its rect.
        # returns a surface representing the ship
        self.image = pygame.image.load('images/ship.bmp') 
        
        # access surface's rect attribute (rect = rectangles); allows to treat elements
        # as rectangles which are simple to deal with; one can access the top, bottom, 
        # left, and right edges, and center
        self.rect = self.image.get_rect() 
        
        # stores screen's rect
        self.screen_rect = screen.get_rect() 
        
        # Start each new ship at the bottom center of the screen.
        # places the x-coordinate of the ship's center to match the center of screen
        self.rect.centerx = self.screen_rect.centerx
        # places the y-coordinate of the ship's bottom to match the bottom of screen
        self.rect.bottom = self.screen_rect.bottom 
        
        # Store a decimal value for the ship's center.
        self.center = float(self.rect.centerx)
        
        # Movement flags
        self.moving_right = False
        self.moving_left = False
    
    def update(self):
        """Update the ship's position based on the movement flags."""
        # Update the ship's center value, not the rect.
        if self.moving_right and self.rect.right < self.screen_rect.right: # limits screen from going off screen
            self.center += self.ai_settings.ship_speed_factor
        if self.moving_left and self.rect.left > 0: # limits screen from going off left margin of screen
            self.center -= self.ai_settings.ship_speed_factor
            
        # Update rect object from self.center.
        self.rect.centerx = self.center # only integer portion of self.center will be stored in self.rect.centerx
        
    def blitme(self):
        """Draw the ship at its current location."""
        self.screen.blit(self.image, self.rect) # draws image to screen at position specified by self.rect
        
    def center_ship(self):
        """Center the ship on the screen."""
        self.center = self.screen_rect.centerx

In [ ]:
# Creating the Bullet Class
import pygame
from pygame.sprite import Sprite
# When you use sprites, you can group related elements in your 
# game and act on all the grouped elements at once.

class Bullet(Sprite):
    """A Class to manage bullets fired from the ship"""
    
    def __init__(self, ai_settings, screen, ship):
        """Create a bullet object at the ship's current position."""
        super(Bullet, self).__init__() # inherit properly from Sprite
        self.screen = screen
        
        # Create a bullet rect at (0,0) and then set correct position.
        # build bullets rect from scratch
        self.rect = pygame.Rect(0, 0, ai_settings.bullet_width, ai_settings.bullet_height) 
        # makes bullets appear at top center of ship
        self.rect.centerx = ship.rect.centerx
        self.rect.top = ship.rect.top 
        
        # Store the bullet's position as a decimal value. (the y-coordinate)
        self.y = float(self.rect.y)
        
        # Store color and speed settings
        self.color = ai_settings.bullet_color
        self.speed_factor = ai_settings.bullet_speed_factor
        
    def update(self):
        """Move the bullet up the screen."""
        # Update the decimal position of the bullet.
        self.y -= self.speed_factor
        # Update the rect position.
        self.rect.y = self.y
    
    def draw_bullet(self):
        """Draw the bullet to the screen."""
        pygame.draw.rect(self.screen, self.color, self.rect)

In [ ]:
# Game Functions
from time import sleep    
    
# KEYSTROKE EVENTS
def check_keydown_events(event, ai_settings, screen, ship, bullets):
    """Respond to keypresses"""
    if event.key == pygame.K_RIGHT: # check if right arrow key is pressed
        ship.moving_right = True
    elif event.key == pygame.K_LEFT: # check if left arrow key is pressed
        ship.moving_left = True
    elif event.key == pygame.K_SPACE:
        fire_bullet(ai_settings, screen, ship, bullets)
    elif event.key == pygame.K_q:
        sys.exit()
        
            
def check_keyup_events(event, ship):
    """Respond to key releases"""
    if event.key == pygame.K_RIGHT: # check if right arrow key is pressed
        ship.moving_right = False
    elif event.key == pygame.K_LEFT: # check if left arrow key is pressed
        ship.moving_left = False

        
def check_events(ai_settings, screen, stats, sb, 
                 play_button, ship, aliens, bullets):
    """
    Respond to keypressed and mouse events.
    stats -> to access the game_active flag
    play_button -> to check whether the Play button has been clicked
    """
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()
        elif event.type == pygame.KEYDOWN: # detects a keydown event
            check_keydown_events(event, ai_settings, screen, ship, bullets)
        elif event.type == pygame.KEYUP: # detects a keyup event    
            check_keyup_events(event, ship)
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouse_x, mouse_y = pygame.mouse.get_pos() # Tuple of where the mouse click is made
            check_play_button(ai_settings, screen, stats, sb, play_button, 
                              ship, aliens, bullets, mouse_x, mouse_y)
            
def check_play_button(ai_settings, screen, stats, sb,
                      play_button, ship, aliens, bullets, mouse_x, mouse_y):
    """Start a new game when the player clicks Play."""
    button_clicked = play_button.rect.collidepoint(mouse_x, mouse_y)
    # checks to see if game was inactive and button has been clicked
    if button_clicked and not stats.game_active:
        # Reset the game settings.
        ai_settings.initialize_dynamic_settings()
        
        # Hide the mouse cursor.
        pygame.mouse.set_visible(False)
        
        # Reset the game statistics.
        stats.reset_stats()
        stats.game_active = True
        
        # Reset the scoreboard images.
        sb.prep_score()
        sb.prep_high_score()
        sb.prep_level()
        sb.prep_ships()
        
        # Empty the list of aliens and bullets
        aliens.empty()
        bullets.empty()
        
        # Create a new fleet and center the ship.
        create_fleet(ai_settings, screen, ship, aliens)
        ship.center_ship()

# UPDATES
def update_screen(ai_settings, screen, stats, sb, ship, aliens, bullets, play_button):
    """Update images on the screen and flip to the new screen."""
    
    # Redraw the screen during each pass through the loop.
    screen.fill(ai_settings.bg_color)
    
    # Redraw all bullets behind ship and aliens.
    for bullet in bullets.sprites(): # bullets.sprites() method returns a list of all sprites in the group bullets
        bullet.draw_bullet() # method from Bullet class
        
    # Redraw ship onscreen after filling the background
    ship.blitme()
    
    # Redraw alien onscreen after filling the background
    # Pygame automatically draws each element in the group at the position defined
    # by its rect attribute
    aliens.draw(screen)
    
    # Draw the score information.
    sb.show_score()
    
    # Draw the play button if the game is inactive
    if not stats.game_active:
        play_button.draw_button()
    
    # Make the most recently drawn screen visible.
    pygame.display.flip()

def update_bullets(ai_settings, screen, stats, sb, ship, aliens, bullets):
    """Update position of bullets and get rid of old bullets."""
    # Update bullet positions
    bullets.update() # calls bullet.update() for each bullet we place in the group bullets
        
    # Get rid of bullets that have disappeared.
    for bullet in bullets.copy(): # you shouldn't remove items from a list or group within a for loop
        if bullet.rect.bottom <= 0:
            bullets.remove(bullet)
            
    check_bullet_alien_collisions(ai_settings, screen, stats, sb, ship, aliens, bullets)
    
def check_bullet_alien_collisions(ai_settings, screen, stats, sb, ship, aliens, bullets):
    "Respond to bullet-alien collisions."
    # Check for any bullets that have hit aliens.
    # If so, get rid of the bullet and the alien.
    # Returns a key-value pair of bullet and alien; a dictionary :P
    # True arguments tell Pygame whether to delete the bullets and aliens that have collided
    collisions = pygame.sprite.groupcollide(bullets, aliens, True, True) 
    if collisions:
        for aliens in collisions.values():
        # Add points when an alien is killed and update score
            stats.score += ai_settings.alien_points * len(aliens)
            sb.prep_score()
        # See if current score beats high score
        check_high_score(stats, sb)
    
    # check to see if any aliens exist
    if len(aliens) == 0:
        # If the entire fleet is destroyed, start a new level
        # Destroy existing bullets
        bullets.empty()
        # Speed up game
        ai_settings.increase_speed()
        
        # Increase level
        stats.level += 1
        sb.prep_level()
        
        # Create new fleet
        create_fleet(ai_settings, screen, ship, aliens)
        
            
def fire_bullet(ai_settings, screen, ship, bullets):
    """Fire a bullet if limit not reached yet"""
    # Create a new bullet and add it to the bullets group.
    if len(bullets) < ai_settings.bullets_allowed: 
        new_bullet = Bullet(ai_settings, screen, ship) # creates instance of bullet
        bullets.add(new_bullet)
        
def get_number_aliens_x(ai_settings, alien_width):
    """Determine the number of aliens that fit in a row."""
    # Calculate horizontal space available for aliens
    available_space_x = ai_settings.screen_width - 2 * alien_width
    # Number of aliens that can fit in that space
    number_aliens_x = int(available_space_x / (2 * alien_width))
    return number_aliens_x

def get_number_rows(ai_settings, ship_height, alien_height):
    """Determine the number of rows of aliens that fit on the screen."""
    available_space_y = (ai_settings.screen_height - (3 * alien_height) - ship_height)
    number_rows = int(available_space_y / (2 * alien_height))
    return number_rows

def create_alien(ai_settings, screen, aliens, alien_number, row_number):
    """Create an alien and place it in the row"""
    alien = Alien(ai_settings, screen)
    alien_width = alien.rect.width
    alien.x = alien_width + 2 * alien_width * alien_number
    alien.rect.y = alien.rect.height + 2 * alien.rect.height * row_number
    alien.rect.x = alien.x
    aliens.add(alien)
        
def create_fleet(ai_settings, screen, ship, aliens):
    """Create a full fleet of aliens."""
    # Create an initial alien and find the number of aliens in a row.
    alien = Alien(ai_settings, screen)
    number_aliens_x = get_number_aliens_x(ai_settings, alien.rect.width)
    number_rows = get_number_rows(ai_settings, ship.rect.height, alien.rect.height)
    
    # Create the first row of aliens
    for row_number in range(number_rows):
        for alien_number in range(number_aliens_x):
            create_alien(ai_settings, screen, aliens, alien_number, row_number)
    
def check_fleet_edges(ai_settings, aliens):
    """Respond appropriately if any aliens have reached an edge."""
    for alien in aliens.sprites():
        if alien.check_edges():
            change_fleet_direction(ai_settings, aliens)
            break

def change_fleet_direction(ai_settings, aliens):
    """Drop the entire fleet and change the fleet's direction"""
    for alien in aliens.sprites():
        alien.rect.y += ai_settings.fleet_drop_speed
    # Change direction
    ai_settings.fleet_direction *= -1

def check_aliens_bottom(ai_settings, screen, stats, sb, ship, aliens, bullets):
    """Check if any aliens have reached the bottom of the screen."""
    screen_rect = screen.get_rect()
    for alien in aliens.sprites():
        if alien.rect.bottom >= screen_rect.bottom:
            # Treat this the same as if the ship got hit
            ship_hit(ai_settings, screen, stats, sb, ship, aliens, bullets)
            break
    
def update_aliens(ai_settings, screen, stats, sb, ship, aliens, bullets):
    """Check if the fleet is at an edge and update the positions of all aliens in the fleet."""
    check_fleet_edges(ai_settings, aliens)
    aliens.update()
    
    # Look for alien-ship collisions.
    # Loops through the group aliens and returns the first alien it finds that has collided with ship
    if pygame.sprite.spritecollideany(ship, aliens):
        ship_hit(ai_settings, screen, stats, sb, ship, aliens, bullets)
        print("Ship hit!!!")
        
    # Look for aliens hitting the bottom of the screen.
    check_aliens_bottom(ai_settings, screen, stats, sb, ship, aliens, bullets)

def ship_hit(ai_settings, screen, stats, sb, ship, aliens, bullets):
    """Respond to ship being hit by alien."""
    if stats.ships_left > 0:
        # Decrement ships_left
        stats.ships_left -= 1
        
        # Update scoreboard.
        sb.prep_ships()
        
        # Empty the list of aliens and bullets.
        aliens.empty()
        bullets.empty()

        # Create a new fleet and center the ship.
        create_fleet(ai_settings, screen, ship, aliens)
        ship.center_ship()

        # Pause.
        sleep(0.5)
        
    # End game if you run out of lives
    else:
        stats.game_active = False
        pygame.mouse.set_visible(True)
    
def check_high_score(stats, sb):
    """Check to see if there's a new high score."""
    if stats.score > stats.high_score:
        stats.high_score = stats.score
        sb.prep_high_score()

In [ ]:
# Creating the Alien Class
import pygame
from pygame.sprite import Sprite

class Alien(Sprite):
    """A class to represent a single alien in the fleet."""
    
    def __init__(self, ai_settings, screen):
        """Initalize the alien and set its starting position."""
        super(Alien, self).__init__() # inherit properly from Sprite
        self.screen = screen
        self.ai_settings = ai_settings
        
        # Load the alien image and set its rect attribute
        self.image = pygame.image.load('images/alien.bmp')
        self.rect = self.image.get_rect()
        
        # Start each new alien near the top left of the screen.
        self.rect.x = self.rect.width
        self.rect.y = self.rect.height
        
        # Store the alien's exact position.
        self.x = float(self.rect.x)
        
    def blitme(self):
        """Draw the alien at its current location."""
        self.screen.blit(self.image, self.rect) # draws image to screen at position specified by self.rect
        
    def check_edges(self):
        """Return True if alien is at edge of screen"""
        screen_rect = self.screen.get_rect()
        # At right edge
        if self.rect.right >= screen_rect.right:
            return True
        # At left edge
        elif self.rect.left <= 0:
            return True
        
    def update(self):
        """Move the alien left or right."""
        # Upon every update, move alien's position to the right by the amount of alien_speed_factor
        self.x += (self.ai_settings.alien_speed_factor * self.ai_settings.fleet_direction)
        
        # Update rect object
        self.rect.x = self.x

    
        
    


In [ ]:
# Creating a Button Class
import pygame.font

class Button():
    
    def __init__(self, ai_settings, screen, msg):
        """Initialize button attributes."""
        self.screen = screen
        self.screen_rect = screen.get_rect()
        
        # Set the dimensions and properties of the button.
        self.width, self.height = 200, 50
        self.button_color = (0,255,0)
        self.text_color = (255,255,255)
        self.font = pygame.font.SysFont(None, 48) # None -> use default font; 48 -> size of font
        
        # Build the button's rect object and center it
        self.rect = pygame.Rect(0, 0, self.width, self.height)
        self.rect.center = self.screen_rect.center
        
        # The button message needs to be prepped only once.
        self.prep_msg(msg)
        
    def prep_msg(self, msg):
        """Turn msg into a rendered image and center text on the button."""
        self.msg_image = self.font.render(msg, True, self.text_color, self.button_color) # True <- turns on antialiasing
        self.msg_image_rect = self.msg_image.get_rect()
        self.msg_image_rect.center = self.rect.center
        
    def draw_button(self):
        # Draw blank button and then draw message.
        self.screen.fill(self.button_color, self.rect) # draw the rectangular portion of the button
        self.screen.blit(self.msg_image, self.msg_image_rect) # draw the text

In [ ]:
# Displaying the Score
import pygame.font
from pygame.sprite import Group


class Scoreboard():
    """A class to report scoring information."""
    
    def __init__(self, ai_settings, screen, stats):
        """Initialize scorekeeping attributes"""
        self.screen = screen
        self.screen_rect = screen.get_rect()
        self.ai_settings = ai_settings
        self.stats = stats
        
        # Font settings for scoring information.
        self.text_color = (30,30,30)
        self.font = pygame.font.SysFont(None, 48)
        
        #Prepare the initial UI images.
        self.prep_score()
        self.prep_high_score()
        self.prep_level()
        self.prep_ships()
        
    def prep_score(self):
        """Turn the score into a rendered image."""
        # rounds numbers to the nearest multiple of 10, 100, 1000
        rounded_score = int(round(self.stats.score, -1)) 
        # String formatting directive, insert commas into numbers when converting a numerical value to a string
        score_str = "Score " + "{:,}".format(rounded_score)
        # Create image
        self.score_image = self.font.render(score_str, True, 
                                            self.text_color, self.ai_settings.bg_color)
        
        # Display the score at the top right of the screen.
        self.score_rect = self.score_image.get_rect()
        self.score_rect.right = self.screen_rect.right - 20
        self.score_rect.top = 20
        
    def prep_high_score(self):
        """Turn the high score into a rendered image."""
        high_score = int(round(self.stats.score, -1))
        high_score_str = "High Score " "{:,}".format(high_score)
        # Create image
        self.high_score_image = self.font.render(high_score_str, True, 
                                                 self.text_color, self.ai_settings.bg_color)
        
        # Center the high score at the top of the screen
        self.high_score_rect = self.high_score_image.get_rect()
        self.high_score_rect.centerx = self.screen_rect.centerx
        self.high_score_rect.top = self.score_rect.top
        
    def prep_level(self):
        """Turn the level into a rendered image."""
        self.level_image = self.font.render("Level: " + str(self.stats.level), True,
                                            self.text_color, self.ai_settings.bg_color)
        
        # Position the level below the score.
        self.level_rect = self.level_image.get_rect()
        self.level_rect.right = self.score_rect.right
        self.level_rect.top = self.score_rect.bottom + 10 # set 10 pixels below score
    
    def prep_ships(self):
        """Show how many ships are left."""
        self.ships = Group() # Will hold ship instances
        for ship_number in range (self.stats.ships_left):
            ship = Ship(self.ai_settings, self.screen) # Create a new ship
            ship.rect.x = 10 + ship_number * ship.rect.width
            ship.rect.y = 10
            self.ships.add(ship)
    
    def show_score(self):
        """Draw score to the screen."""
        self.screen.blit(self.score_image, self.score_rect) # object, location
        self.screen.blit(self.high_score_image, self.high_score_rect)
        self.screen.blit(self.level_image, self.level_rect)
        
        # Draw ships.
        self.ships.draw(self.screen)
        

In [ ]:
# Game Stats class
class GameStats():
    """Track statistics for Alien Invasion."""
    
    def __init__(self, ai_settings):
        """Initialize statistics."""
        self.ai_settings = ai_settings
        self.reset_stats()
        
        # Start Alien Invasion in an inactive state.
        self.game_active = False
        
        # High score should never be reset.
        self.high_score = 0
    
    def reset_stats(self):
        """Initialize statistics that can change during the game."""
        self.ships_left = self.ai_settings.ship_limit
        self.score = 0
        self.level = 1
        